In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def display_images(images_data, nrows=10, ncols=8):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16 * nrows, 16 * ncols))
    axes = axes.reshape(nrows, ncols)
    for i_row, row_axes in enumerate(axes):
        for j, ax in enumerate(row_axes):
            ax.imshow(images_data[i_row * len(row_axes) + j].reshape(56, 56, 3).astype(np.uint8))

In [3]:
train_images_fname = 'Data/db_train_balanced.raw'
train_labels_fname = 'Data/label_train_balanced.txt'
# val_images_fname = 'Data/db_val.raw'
# val_labels_fname = 'val_pred.txt'
# number of images
num_train_images = 30108
num_valid_images = 10130

image_dim = 56 * 56 * 3

In [ ]:
balanced_data, balanced_labels = load_data(num_train_images, image_dim, 0, 100)
balanced_data, balanced_labels = balance(balanced_data, balanced_labels)
balanced_data2, balanced_labels2 = load_data(num_train_images, image_dim, 100, 200)
balanced_data2, balanced_labels2 = balance(balanced_data2, balanced_labels2)

all_data = []
all_data.append(balanced_data)
all_data.append(balanced_data2)

all_labels = []
all_labels.append(balanced_labels)
all_labels.append(balanced_labels2)

final_data = np.vstack(all_data)
final_labels = np.hstack(all_labels)

print(final_data.shape)
print(final_labels.shape)

In [ ]:
display_images(final_data[final_labels == 1], 4, 4)

In [ ]:
display_images(final_data[final_labels == 0])

In [5]:
num_train_images = 111430
image_dim = 56 * 56 * 3
total_data_length = num_train_images * image_dim
ten_percent = int(num_train_images / 10)

def load_data(num_train_images, image_dim, start, end):
    return images_data[start:end], labels[start:end]

def balance(data, labels):
    data_and_labels = np.hstack((data, labels.reshape(-1, 1)))
    zeros = data_and_labels[labels == 0]
    ones = data_and_labels[labels == 1]
    ones = ones[:len(zeros)]
    data_and_labels = np.vstack((zeros, ones))
    return data_and_labels[:, :data.shape[1]], data_and_labels[:, -1]    

def load_and_balance(input_data, input_labels):
    start = 0
    end = ten_percent
    all_data = []
    all_labels = []
    for i in range(10):
        data, labels = input_data[start:end], input_labels[start:end]
        data, labels = balance(data, labels)
        print(data.shape)
        all_data.append(data)
        all_labels.append(labels)
        start += ten_percent
        end += ten_percent

    all_data = np.vstack(all_data)
    all_labels = np.hstack(all_labels).reshape(-1, 1)
    return all_data, all_labels

def shuffle(data, labels):
    nb_data = data.shape[0]
    arr = np.arange(len(data))
    np.random.shuffle(arr)
    tmp_data = data[arr[0], :]
    for i in range(nb_data - 1):
        data[arr[i], :] = data[arr[i + 1], :]
    data[arr[nb_data - 1], :] = tmp_data
    
    tmp_label = labels[arr[0], :]
    for i in range(nb_data - 1):
        labels[arr[i], :] = labels[arr[i + 1], :]
    labels[arr[nb_data - 1], :] = tmp_label

    return data, labels
    
def save_to_disk(data, labels):
    data = data.reshape(1, -1)[0]
    data.astype(np.uint8).tofile('Data/db_train_balanced2.raw')
    np.savetxt('Data/label_train_balanced2.txt', labels)

    
with open('Data/db_train.raw', 'rb') as f:
    data = np.fromfile(f, dtype=np.uint8, count=num_train_images * image_dim)
    data = data.reshape(num_train_images, image_dim)
labels = np.loadtxt('Data/label_train.txt', dtype=np.float64)
print("labels shape = " + str(labels.shape))
print("data shape = " + str(data.shape))
data, labels = load_and_balance(data, labels)
print("labels shape = " + str(labels.shape))
print("data shape = " + str(data.shape))
data, labels = shuffle(data, labels)
print("labels shape = " + str(labels.shape))
print("data shape = " + str(data.shape))
save_to_disk(data, labels)
print("labels shape = " + str(labels.shape))
print("data shape = " + str(data.shape))

labels shape = (111430,)
data shape = (111430, 9408)
(3046, 9408)
(3030, 9408)
(2968, 9408)
(3022, 9408)
(3086, 9408)
(3088, 9408)
(3104, 9408)
(3046, 9408)
(3206, 9408)
(2904, 9408)
labels shape = (30500, 1)
data shape = (30500, 9408)
labels shape = (30500, 1)
data shape = (30500, 9408)
labels shape = (30500, 1)
data shape = (30500, 9408)


In [6]:
print(data.dtype)

float64


In [7]:
test_labels = np.loadtxt('Data/label_train_balanced2.txt', dtype=np.float64)

test_labels.shape

(30500,)

In [ ]:
def load_split_save(raw_name, first_half_name, second_half_name, num_images):
    # size of the images 48*48 pixels in gray levels
    image_dim = 56 * 56 * 3
    with open(raw_name, 'rb') as f:
        images_data = np.fromfile(f, dtype=np.uint8, count=num_images * image_dim)
        half_length = int(images_data.shape[0]*0.5)
        first_images_data = images_data[:half_length]
        first_images_data.tofile(first_half_name)
        second_images_data = images_data[half_length:]
        second_images_data.tofile(second_half_name)

load_split_save('Data/db_train.raw', 'Data/db_train_first_half.raw', 'Data/db_train_second_half.raw', 111430)

In [3]:
train_images_fname = 'Data/db_train_balanced2.raw'
train_labels_fname = 'Data/label_train_balanced2.txt'
val_images_fname = 'Data/db_val.raw'
val_labels_fname = 'val_pred.txt'
# number of images
num_train_images = 30500
num_valid_images = 10130

use_train = False
if use_train:
    data_fname = train_images_fname
    labels_fname = train_labels_fname
    nb_data = num_train_images
else:
    data_fname = val_images_fname
    labels_fname = val_labels_fname
    nb_data = num_valid_images

# size of the images 48*48 pixels in gray levels
image_dim = 56 * 56 * 3

labels = np.loadtxt(labels_fname, dtype=np.float64)

with open(data_fname, 'rb') as f:
    data = np.fromfile(f, dtype=np.uint8, count=nb_data * image_dim).astype(np.float32)
    data = data.reshape(nb_data, image_dim)

# with open(val_images_fname, 'rb') as f:
#     val_images_data = np.fromfile(f, dtype=np.uint8, count=num_valid_images * image_dim).astype(np.float32)
#     val_images_data = val_images_data.reshape(num_valid_images, image_dim)

In [4]:
print(labels.shape)
print(data.shape)
print(data[labels == 0].shape)
print(data[labels == 1].shape)

(10130,)
(10130, 9408)
(2001, 9408)
(8129, 9408)


In [6]:
labels0 = np.loadtxt('val_pred_0', dtype=np.float64)
labels1 = np.loadtxt('val_pred_1', dtype=np.float64)

print(labels0.shape)
print(labels1.shape)

(100,)
(100,)


In [8]:
def display_label(data, labels, value):
    display_images(data[labels == value])

display_label(data[:100], labels0, 1)

In [ ]:
# Performance measure
def compute_pred_score(y_true, y_pred):
    if (y_true.shape[0] != y_pred.shape[0]):
        raise Exception("y_pred and y_true must have the same size")
    tmp = np.unique(y_pred)
    if (tmp.shape[0] > 2):
        raise("y_pred should only contain 0 and 1")
        
    if (tmp.shape[0] == 1):
        if (tmp[0] != 0 and tmp[0] != 1):
            raise Exception("y_pred should only contain 0 and 1")
    else:
        if (tmp[0] != 0 and tmp[1] != 1):
            raise Exception("y_pred should only contain 0 and 1")
    
    y0 = (y_true == 0)
    y1 = (y_true == 1)
    
    acc1 = np.mean(1 - y_pred[y0])
    acc2 = np.mean(y_pred[y1])
    
    return (acc1+acc2)/2

In [ ]:
# Log. regression on the first 1000 elements of the training dataset
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(train_images_data[:1000,:], train_images_label[:1000])  

tmp = clf.predict(train_images_data)
acc_train = compute_pred_score(train_images_label, tmp)
print('The score on the training data : %s' % acc_train)

In [ ]:
# Generate the prediction
val_pred = clf.predict(val_images_data)
np.savetxt("val_pred.txt", val_pred, fmt="%d")